In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sqlite3
import sys
from pathlib import Path

In [ ]:
CHROME_QUERY  = """SELECT url, title, datetime((last_visit_time/1000000)-11644473600, 'unixepoch', 'localtime') 
                   AS last_visit_time FROM urls ORDER BY last_visit_time DESC"""

FIREFOX_QUERY = """SELECT url, title, datetime((visit_date/1000000), 'unixepoch', 'localtime') AS visit_date 
                   FROM moz_places INNER JOIN moz_historyvisits on moz_historyvisits.place_id = moz_places.id ORDER BY visit_date DESC"""

SAFARI_QUERY  = """SELECT url, title, datetime(visit_time + 978307200, 'unixepoch', 'localtime') 
                   FROM history_visits INNER JOIN history_items ON history_items.id = history_visits.history_item ORDER BY visit_time DESC"""


In [ ]:
OSX, WINDOWS, LINUX = ["darwin"], ["cygwin", "win32"], ["linux", "linux2"]
HOME = Path.home()

class Importer():
    def __init__(self): pass

    def read_db(self, path, query):
        conn   = sqlite3.connect(str(path))
        cursor = conn.cursor()
        cursor.execute(query)
        query_result = cursor.fetchall()
        cursor.close()
        conn.close()
        return query_result

In [ ]:
CHROME_OSX     = HOME / "Library" / "Application Support" / "Google" / "Chrome" / "Default" / "History"
CHROME_WINDOWS = HOME / "AppData" / "Local" / "Google" / "Chrome" / "User Data" / "Default" / "History"
CHROME_LINUX   = HOME / ".config" / "google-chrome" / "Default" / "History"

class ChromeImporter(Importer):
    
    def __init__(self):
        super().__init__()
          
    def get_path():
        if sys.platform   in OSX    : return CHROME_OSX 
        elif sys.platform in WINDOWS: return CHROME_WINDOWS
        elif sys.platform in LINUX:   return CHROME_LINUX
        
    def get_data(self):
        return read_db(self.get_path(), CHROME_QUERY)

In [ ]:
FIREFOX_OSX     = HOME / "Library" / "Application Support" / "Firefox" / "Profiles"
FIREFOX_WINDOWS = HOME / "AppData" / "Roaming" / "Mozilla" / "Firefox" / "Profiles"
FIREFOX_LINUX   = HOME / ".mozilla" / "firefox"

class FireFoxImporter(Importer):
    def __init__(self):
        super().__init__()
        
    def get_path(self):
        if sys.platform in OSX:       abs_path = FIREFOX_OSX
        elif sys.platform in WINDOWS: abs_path = FIREFOX_WINDOWS
        elif sys.platform in LINUX:   abs_path = FIREFOX_LINUX
        return find_firefox_path(abs_path)
    
    def find_firefox_path(self, abs_firefox_path):
        # searches for a directory that ends ".default"
        for path in abs_firefox_path.ls():
            if str(path).find(".default") > 0: return path / "places.sqlite"
            
    def get_data(self):
        return read_db(self.get_path(), FIREFOX_QUERY)

In [ ]:
SAFARI_OSX = HOME / "Library" / "Safari" / "History.db"

class SafariImporter(Importer):
    
    def __init__(self):
        super().__init__()
    
    def get_path(self):
        if sys.platform in OSX: return SAFARI_OSX
        else:
            print("Safari only supported on mac")
            return None
        
    def get_data(self):
        if not sys.platform in OSX:
            print("Safari only supported on mac")
            return []
        return read_db(self.get_path(), SAFARI_QUERY)

In [ ]:
BRAVE_OSX     = HOME / "Library" / "Application Support" / "BraveSoftware" / "Brave-Browser" / "Default" / "History"
BRAVE_WINDOWS = HOME / "AppData" / "Local" / "BraveSoftware" / "Brave-Browser" / "User Data" / "Default" / "History"
BRAVE_LINUX   = HOME / ".config" / "BraveSoftware" / "Brave-Browser" / "Default" / "History"

class BraveImporter(Importer):
    
    def __init__(self):
        super().__init__()
          
    def get_path(self):
        if sys.platform   in OSX:     return BRAVE_OSX
        elif sys.platform in WINDOWS: return BRAVE_WINDOWS
        elif sys.platform in LINUX:   return BRAVE_LINUX
        
    def get_data(self):
        return self.read_db(self.get_path(), CHROME_QUERY)

In [ ]:
class URLVisit():
    
    def __init__(self, url, title, time):
        self.url=url
        self.title=title
        self.time=time
        
    def __repr__(self):
        return f"\nURLVisit\nurl: {self.url}\ntitle:{self.title}\ntime:{self.time}\n"

In [ ]:
importer = BraveImporter()

In [ ]:
history = [URLVisit(url, title, time) for url, title, time in importer.get_data()]

In [ ]:
print(history[100:105])

[
URLVisit
url: https://github.com/googleapis/google-api-python-client/raw/master/docs/oauth-server.md
title:
time:2020-12-21 12:11:24
, 
URLVisit
url: https://raw.githubusercontent.com/googleapis/google-api-python-client/master/docs/oauth-server.md
title:
time:2020-12-21 12:11:24
, 
URLVisit
url: https://developers.google.com/accounts/docs/OAuth2ServiceAccount
title:Using OAuth 2.0 for Server to Server Applications  |  Google Identity
time:2020-12-21 12:09:59
, 
URLVisit
url: https://tools.ietf.org/html/rfc6749
title:RFC 6749 - The OAuth 2.0 Authorization Framework
time:2020-12-21 12:07:24
, 
URLVisit
url: https://github.com/googleapis/google-api-python-client/blob/master/docs/pagination.md
title:google-api-python-client/pagination.md at master · googleapis/google-api-python-client
time:2020-12-21 12:06:21
]
